This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=1, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = QallseD0 # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
#metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, **dsmaker_config)
metas, path = create_dataset(input_path='/Users/parkerreid/PycharmProjects/ATLASread/Dat/ATLAS_TrackML-event69927-hits.csv'\
                             ,output_path=tempdir.name, gen_doublets=True, high_pt_cut = 0)

03:35:01.729 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 383 hits from /Users/parkerreid/PycharmProjects/ATLASread/Dat/ATLAS_TrackML-event69927.
03:35:01.747 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 151.
/Users/parkerreid/QMHEP/lib/python3.7/site-packages/pandas-0.23.4-py3.7-macosx-10.14-x86_64.egg/pandas/core/indexing.py:189: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/dsmaker.py:167: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/parkerreid/QMHEP/lib/python3.7/site-packages/pandas-0.23.4-py3.7-macosx-10.14-x86_64.egg/pandas/core/generic.py:4405: Setting

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpjg1u5_dz
                    hit_id       x       y        z   truth  volume_id  \
hit_id                                                                   
8.751523e+16  8.751523e+16  134.19  130.80 -1137.00  1819.0        8.0   
2.895765e+17  2.895765e+17   31.44   21.15   -74.47  1064.0        8.0   
2.913365e+17  2.913365e+17  -11.61   36.53    69.57  1522.0        8.0   
2.935270e+17  2.935270e+17  -36.29  -13.20   -49.39   973.0        8.0   
2.941522e+17  2.941522e+17  -26.32  -29.67    80.71   406.0        8.0   

              layer_id  module_id       hit_id_  particle_id    tx    ty  \
hit_id                                                                     
8.751523e+16       2.0        2.0  8.751523e+16       1819.0 -0.01 -0.01   
2.895765e+17       0.0        2.0  2.895765e+17       1064.0 -0.00 -0.01   
2.913365e+17       0.0        2.0  2.913365e+17       1522.0 -0.03  0.02   
2.935270e+17       0.0        2.0 

03:35:01.988 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=1) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmpjg1u5_dz/ez-0.1_baby/event69927.


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
[4 5]
[4 5]
[4 5]
[4 5]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
[3]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
[1]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
[6]
[7]
0
0
0
0
0
0
0
0
8
[8]
[9]
[10]
0
0
0
0
0
0
0
0
11
[11]
[12]
0
0
0
0
0
0
0
0
13
[13]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
[2]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[0]
0
0
0
0
0
0
0
0


In [4]:
#with open(path + '-meta.json') as f:
#    print(f.read())

# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 1 doublets.
  Input precision (%): 200.0000, recall (%): 40.0000
    New precision (%): 150.0000


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
Q = model.to_qubo()

03:35:02.057 DEBUG hepqpr.qallse.qallse_d0: using config:
03:35:02.059 DEBUG hepqpr.qallse.qallse_d0:     beamspot_center: (0, 0, 0)
03:35:02.059 DEBUG hepqpr.qallse.qallse_d0:     beamspot_width: 27.5
03:35:02.061 DEBUG hepqpr.qallse.qallse_d0:     cheat: False
03:35:02.062 DEBUG hepqpr.qallse.qallse_d0:     d0_denom: 1.0
03:35:02.063 DEBUG hepqpr.qallse.qallse_d0:     d0_factor: 0.5
03:35:02.064 DEBUG hepqpr.qallse.qallse_d0:     max_layer_span: 2
03:35:02.065 DEBUG hepqpr.qallse.qallse_d0:     min_qplet_path: 2
03:35:02.065 DEBUG hepqpr.qallse.qallse_d0:     num_multiplier: -1
03:35:02.066 DEBUG hepqpr.qallse.qallse_d0:     qplet_max_dcurv: 0.0001
03:35:02.066 DEBUG hepqpr.qallse.qallse_d0:     qplet_max_strength: -0.2
03:35:02.067 DEBUG hepqpr.qallse.qallse_d0:     qubo_bias_weight: 0
03:35:02.069 DEBUG hepqpr.qallse.qallse_d0:     qubo_conflict_strength: 1
03:35:02.070 DEBUG hepqpr.qallse.qallse_d0:     rz_power: 1
03:35:02.071 DEBUG hepqpr.qallse.qallse_d0:     strength_bounds: N

CPU times: user 41.2 ms, sys: 9.33 ms, total: 50.6 ms
Wall time: 50.2 ms


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
response = model.sample_qubo(Q=Q)

AttributeError: 'NoneType' object has no attribute 'sample_qubo'

## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample(next(response.samples()))
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

NameError: name 'response' is not defined

# Evaluate the results

## Print statistics and scores

In [ ]:
# stats about the qbsolv run
en0 = dw.compute_energy(Q)
en = response.record.energy[0]

print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
occs = response.record.num_occurrences
print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [ ]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [ ]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [ ]:
iplot_results_tracks(dw, final_tracks, dims=list('zxy'))